#### Aerocet Sampling Data, using kml files and pandas

- The data from dust pollution monitoring in Coimbatore using Aerocet was copy paste into a csv file from miniterm.py program and command ```2``` given to aerocet. Seprate CSV files were made for count and mass mode sampling. 
-  The location of sampling point is made into a point feature in Google earth with name of respective sampling time. This was saved as kml file and opened in QGIS converted into shape file to add gemoetry. The shape file was subject to edit by attribute calculator and added new two columns of lat and lon by x and y gemorty for respective columns, these columns were formated as STRINGS to get all decimal values of lattitue and longitude. The shape files is then converted into csv files using MMQGS plugin. Giving decimal or real number rounds the lat long values and become non usefull while converting into CSV file. 
-  The csv files two of aerocet data with time and location data (lat lon) with time have to merge into single file for storing and further analysis. 
-  An initial attempt was made with [this](http://stackoverflow.com/questions/20414562/python-joining-csv-files-where-key-is-first-column-value) to merge the csv files as dcits, but having limitations in terms of sorting, join/merge and other sql function needed for analysis. Then found familar pandas libarary is most useful in this case. 
-  To join location file with nay of the data file, the time has to smilar fomrate. In location file only time is given, in data file it is date and time. A huge number of search and attempts were made to do text into column for data file time column having data and time seprated by sapce. Based on [this](http://stackoverflow.com/questions/18034361/how-to-split-a-dataframe-column-into-multiple-columns), [this](http://stackoverflow.com/questions/14745022/pandas-dataframe-how-do-i-split-a-column-into-two) and most handy by [this](http://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-columns), but having problem in the commands 

```bash
# to import csv file
import pandas as pd
df = pd.read_csv(PMCount_08102014.csv)
df1 = pd.DataFrame(df.Time.str.split(' ',1).tolist(),columns = ['Date','Time'])
```
returns a new separate dataframe with splitter columns, the above cocmmand is based on [this](http://stackoverflow.com/questions/12504976/get-last-column-after-str-split-operation-on-column-in-pandas-dataframe?lq=1). It becomed difccult in join again these data frame with mother data frame. Faced problem such as ```AssertionError: length of join_axes must not be equal to 1``` for doing merge those data frame by merge as ```df1 = pd.merge(df, s, on='Time', suffixes=['Time','Status']), and for doing concat by ```df1 = pd.concat([df,s],join_axes='Time')``` getting ```ValueError: too many values to unpack``` 
-  Then made attempt of reciprocal method that is add date to the place csf file data frame and join with other data file. For this to add date to the time data frame, used this ```df['Name']='2014-10-08 ' + df['Name'].astype(str)```, here Name was the column heading given for the time as it is given in kml file. this column heading was changed by ```df=df.rename(columns={'Name':'Time'})```
-  With this also mergeing ends with error of ```AssertionError: length of join_axes must not be equal to 1``` from command ``` df2=pd.concat([df,df1],join_axes='Time')```. Then it is found that the error is due to lack of index in two merging data frame. Index for data farme was made by ```df.set_index('Time'); df1.set_index('Time')```,  then merging of the datframe ends without any any error by the command ```df2 = pd.merge(df, df1, on='Time', suffixes=['_1', '_2'])```. Withis all the three dataframe can be merged one by one, in which suffixes gives tag to differentiate the two datframe same columns heading. 
-  With this formed dataframe the removal of unwanted columns is tryed with ```del df4['AT(C)_x']``` having limation of only single columns delete, ```df4.drop(['RH(%)_x','Location_x'],axis=1)``` doing multi column delte but giving unsatisfactory, poping up of droped columns in latter commnads, then it is found that make a new data frame with only requiered column list as by below command 
```python
df5=pa.DataFrame(df,columns=['Time','lon','lat','0.3(CF)', '0.5(CF)','1.0(CF)', '5.0(CF)','10(CF)','PM1(ug/m3)','PM2.5(ug/m3)', 'PM4(ug/m3)','PM7(ug/m3)','PM10(ug/m3)', 'TSP(ug/m3)', 'AT(C)_y','RH(%)_y','Location_y', 'Seconds_y','Status_y'])        
```
- The full commnads are as follows

```python
df1 = pd.read_csv('PMCount_08102014.csv')
df2 = pd.read_csv('PMMass_08102014.csv')
df3 = pd.read_csv('08102014L.csv')
df3['Name']='2014-10-08 ' + df3['Name'].astype(str)
df3=df3.rename(columns={'Name':'Time'})
df1.set_index('Time')
df2.set_index('Time')
df3.set_index('Time')
df4 = pd.merge(df1, df2, on='Time', suffixes=['_1', '_2'])
df5 = pd.merge(df3, df4, on='Time')
df6=pd.DataFrame(df5,columns=['Time','lon','lat','0.3(CF)', '0.5(CF)','1.0(CF)', '5.0(CF)','10 (CF)','PM1(ug/m3)','PM2.5(ug/m3)', 'PM4(ug/m3)','PM7(ug/m3)','PM10(ug/m3)', 'TSP(ug/m3)', 'AT(C)_2','RH(%)_2','Location_2','Seconds_2','Status_2'])
 df6.to_csv('A08102014M.csv')
```
-  kml to csv file was generated by this [scripts](kml-to-csv-master.zip in GITHUB), it convert kml point location file into csv file with name and latitude and longitude column stitched together as a tuple. The kml file gives two coordinates one is with name tag latitude and longitude and another with coordinates. In which coordinates is current lat long as by checked its factual in google earth. below is a example of kml file point coordinates

```xml
<Placemark>
			<name>07:37:03</name>
			<LookAt>
				<longitude>76.94981740084671</longitude>
				<latitude>10.91510509935186</latitude>
				<altitude>0</altitude>
				<heading>0.008824190954161487</heading>
				<tilt>0</tilt>
				<range>830.3193705463046</range>
				<gx:altitudeMode>relativeToSeaFloor</gx:altitudeMode>
			</LookAt>
			<styleUrl>#m_ylw-pushpin</styleUrl>
			<Point>
				<coordinates>76.94856689929749,10.91446297419933,0</coordinates>
			</Point>
</Placemark>
```
- more over the kml to csv script was edited to take only coordiantes and names, this avoid the error comes from locus map point marking kml files where the name tag lot lang is absent. The modified script is as follows

```python
"""
# pip install pykml
pykml depends to lxml module
this script convert only one folder to csv
"""
import sys, csv, codecs
from pykml import parser as kmlParser
def main():
	args = sys.argv[1:]
	
	if 1 > len(args) > 2: usage()
	filename = args[0]
	name, places = getdata(filename)
	lst = places_to_list(places)
	
	if filename.endswith('.kml') : filename = filename.replace('.kml','.csv')
	else: filename += '.csv'
	if len(args) == 2: filename = args[1]
	to_csv(filename, lst)
	print "See {filename}\nGoodluck (^_^)!".format(filename=filename)
def usage():
	print "python parse.py input.mkl output.csv"
	sys.exit(2)
def getDocumentAndTagPrefix(xml):
	root = xml.getroot();
	tagPrefix = root.tag.replace( '}' + xml.docinfo.root_name, '}' )
	return root.Document, tagPrefix

def iterFolders(folder, tagprefix):
	lst = [ i for i in folder.iterchildren(tagprefix+'Folder') ]
	return folder if not len(lst) else iter(lst)
def getdata(filename):
	document, tagPrefix =getDocumentAndTagPrefix( kmlParser.parse( open(filename) ) )
	mainFolder = document.Folder
	name = None
	placemarks = None;
	for folder in iterFolders(mainFolder, tagPrefix):
		name = folder.iterchildren(tagPrefix+"name").next().text
		placemarks = [ x for x in folder.iterchildren(tagPrefix+"Placemark") ]
		break;
	return name,placemarks;
def places_to_list(places):
	lst = [['name', 'coordinates']]
	for place in places:
		lst.append([
			place.name, place.Point.coordinates
		])
	return lst
def to_csv(filename, lst):
	with codecs.open(filename, 'wb') as fl:
		out = csv.writer(fl);	
		out.writerows(lst)
if __name__ == "__main__":
	main()
	sys.exit(0)
```	
-  The modified script gives name and coordinates as separate tuples such as below

```
name,coordinates
18:48:29,"76.938598315386,10.94663518249638,0"
```
-  In Aerocet sampling locus map android application was used with a tablet phone and each point was marked in locus map tracking facility. So for duplicate point sampling the kml file was generated with the point marked with more than one time as names. So the name columns has to be attached with more than one same coordinates in coordinates columns.
-  To do above requirements first tried with simple list split and join like below to open csv as tuples based on [this](http://stackoverflow.com/questions/18776370/converting-a-csv-file-into-a-list-of-tuples-with-python)

```python
with open('21102014_N.csv') as f:
     data=[tuple(line) for line in csv.reader(f)]
#for a sample name and coordinates
location_in = "19:10:29, 19:11:29, 19:12:29","76.96160393798016,10.9879284257414,0"
#to separate the time and location tuples
time    = location_in[0]
loc    = location_in[1]
ts=time.split(', ')
#for loop for attaching name and location different try, #based on [this](http://stackoverflow.com/questions/5998569/python-convert-comma-separated-string-into-reducing-string-list), [this](http://stackoverflow.com/questions/1720421/merge-two-lists-in-python), [this](http://stackoverflow.com/questions/4005796/attributeerror-str-object-has-no-attribute-append), 
for tss in ts:
    a2=str(ts[0])+loc
    a.append(a2)
#based on [this](http://stackoverflow.com/questions/3459098/create-list-of-single-item-repeated-n-times-in-python)
a=[]
for tu in data:
     ts=str(tu[0].split(', '))
     tp=ts[0+','+(len(ts[0])*tu[1])
     a.append(tp)
a=[]
for tu in data:
     ts=tu[0][0]+(len(tu[0][0])*tu[0][1])
     a.append(ts)
#but list based mehtod is not easy to applay, but below #pandas df based solution is doing exactly what is required #based on [this](http://stackoverflow.com/questions/12680754/split-pandas-dataframe-string-entry-to-separate-rows)
df23=pd.concat([Series(row['coordinates'], row['names'].split(',')) for _, row in df.iterrows()]).reset_index()
#to add columns to dataframe
df23.columns = ['names', 'coordinates']
#to split columns in pandas datafarme based on comma #delimiter, for comma separated values into two in #coordinates column. But it is not working
df32 = pd.DataFrame(df3.coordinates.str.split(', ',1).tolist(),columns =['flips','row'])
#to split columns in pandas dataframe, it is working, based #on [this](http://stackoverflow.com/questions/7353353/#trying-to-split-one-list-into-multiple-lists-inside-of-a-#main-list-python)
list=df3['coordinates'].tolist()
the_new_list = [x.split(',') for x in list]
ab=pd.DataFrame(the_new_list)
ab.to_csv('data5.csv')
#to set the column as date time in pandas based on [this](http://stackoverflow.com/questions/16852911/how-do-i-convert-dates-in-a-pandas-data-frame-to-a-date-data-type)
df['time'] = df['time'].astype('datetime64[ns]')
```
-  Other usefull pandas commands
- to preview dataframe based on [this](http://stackoverflow.com/questions/15006298/how-to-preview-a-part-of-a-large-pandas-dataframe) ```df.head()```more info on reshaping and pivoting(grouping) is from [this](http://pandas.pydata.org/pandas-docs/stable/reshaping.html),  [this](http://pandas.pydata.org/pandas-docs/stable/groupby.html) and more and easy intro to pandas is [this](http://www.gregreda.com/2013/10/26/working-with-pandas-dataframes/)